# Sudoku-Solver
---
_**Module**: HSLU - Artificial Intelligence - Search & Optimization (AISO)_  
_**Source**: Slides "Constraint programming 1 - Modelling with OR-Tools"_  
_**Author**: Prof. Dr. Marc Pouly_

>Fill in numbers from 1 to 9 so that each row, column and 3x3 block contains each number exactly once.

![Sudoku](images/Sudoku.png)

## Imports

In [1]:
from ortools.constraint_solver import pywrapcp
import itertools
import numpy as np

## Helper functions

In [2]:
def printSolution(currentSolution):
    # Top Line
    print("╔═══════════╦═══════════╦═══════════╗")
    
    # Field
    for row in range(9):
        newRow = "║ "
        
        for column in range(9):
            newRow += "{}".format(currentSolution[row][column].Value())
            newRow += " ║ " if (column + 1) % 3 == 0 else " │ "            
                                  
        print(newRow)
        
        if((row == 2) or (row == 5)):
            print("╠═══════════╬═══════════╬═══════════╣")
        else:
            if (row != 8):
                print("║───┼───┼───║───┼───┼───║───┼───┼───║");
        
    # Bottom Line
    print("╚═══════════╩═══════════╩═══════════╝\n")

## Solution

In [3]:
solver = pywrapcp.Solver("Sudoku")

# 9x9 Matrix of Decision Variables in {1..9}:
board = [[solver.IntVar(1, 9) for _j in range(9)] for _i in range(9)]

# 9x9 Sudoku to resolve:
sudoku = [
    [4, 0, 8,  0, 0, 0,  0, 0, 0],
    [0, 0, 0,  1, 7, 0,  0, 0, 0],
    [0, 0, 0,  0, 8, 0,  0, 3, 2],
    
    [0, 0, 6,  0, 0, 8,  2, 5, 0],
    [0, 9, 0,  0, 0, 0,  0, 8, 0],
    [0, 3, 7,  6, 0, 0,  9, 0, 0],
    
    [2, 7, 0,  0, 5, 0,  0, 0, 0],
    [0, 0, 0,  0, 1, 4,  0, 0, 0],
    [0, 0, 0,  0, 0, 0,  6, 0, 4]
]

# Feed solver with the predefined Sudoku...
for row in range(9):
    for column in range(9):
        if (sudoku[row][column]) != 0:
            solver.Add(board[row][column] == sudoku[row][column])

            
# --------------------------------------------------------------------------------
# Set constraints
# --------------------------------------------------------------------------------
# Each row/column contains only different values:
for i in range(9):
    solver.Add(solver.AllDifferent([board[i][j] for j in range(9)])) # Rows
    solver.Add(solver.AllDifferent([board[j][i] for j in range(9)])) # Columns

# Tuples (i, j) for all cell indices:
cell_indices = list(itertools.product(range(3), repeat=2))

# Every 3x3 cell contains only different values
for i, j in cell_indices:
    solver.Add(solver.AllDifferent([board[i * 3 + di] [j * 3 + dj] for di in range(3) for dj in range(3)]))

    
# --------------------------------------------------------------------------------
# Now solve it!
# --------------------------------------------------------------------------------
db = solver.Phase(list(np.concatenate(board)), # Sudoku board as array
                solver.INT_VAR_SIMPLE,       # Variable selection policy for search
                solver.INT_VALUE_SIMPLE)     # Value selection policy for search

solver.NewSearch(db)

while solver.NextSolution():
    printSolution(board)

solver.EndSearch()

#Display the number of solutions found:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime: {}ms".format(solver.WallTime()))
print("Failures: {}".format(solver.Failures()))
print("Branches: {}".format(solver.Branches()))

╔═══════════╦═══════════╦═══════════╗
║ 4 │ 2 │ 8 ║ 5 │ 6 │ 3 ║ 1 │ 7 │ 9 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 3 │ 5 │ 9 ║ 1 │ 7 │ 2 ║ 4 │ 6 │ 8 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 7 │ 6 │ 1 ║ 4 │ 8 │ 9 ║ 5 │ 3 │ 2 ║ 
╠═══════════╬═══════════╬═══════════╣
║ 1 │ 4 │ 6 ║ 3 │ 9 │ 8 ║ 2 │ 5 │ 7 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 5 │ 9 │ 2 ║ 7 │ 4 │ 1 ║ 3 │ 8 │ 6 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 8 │ 3 │ 7 ║ 6 │ 2 │ 5 ║ 9 │ 4 │ 1 ║ 
╠═══════════╬═══════════╬═══════════╣
║ 2 │ 7 │ 4 ║ 9 │ 5 │ 6 ║ 8 │ 1 │ 3 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 6 │ 8 │ 3 ║ 2 │ 1 │ 4 ║ 7 │ 9 │ 5 ║ 
║───┼───┼───║───┼───┼───║───┼───┼───║
║ 9 │ 1 │ 5 ║ 8 │ 3 │ 7 ║ 6 │ 2 │ 4 ║ 
╚═══════════╩═══════════╩═══════════╝

Solutions: 1
Runtime: 3ms
Failures: 20
Branches: 40
